
# EDA with SQL — SpaceX Launches (SQLite)

**Author:** Iliya Pezeshki  
**Course:** IBM Data Science Professional Certificate — Capstone  

**Objective:** Load the SpaceX CSV dataset into SQLite and perform EDA-style queries to summarize launch activity, payloads, mission outcomes, and trends.


## 1) Environment Setup

In [10]:
%load_ext sql
import pandas as pd, sqlite3, os

db_path = "my_spacex.db"
with sqlite3.connect(db_path) as con:
    # load CSV
    CSV_URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_2/data/Spacex.csv"
    df = pd.read_csv(CSV_URL)
    df.to_sql("SPACEXTBL", con, if_exists="replace", index=False, method="multi")

# point the sql extension at the DB
%sql sqlite:///my_spacex.db
print("SQLite DB initialized at", os.path.abspath(db_path))

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
SQLite DB initialized at c:\Users\USER\Downloads\my_spacex.db


## 2) Load CSV into SQLite

In [11]:
%%sql
DROP TABLE IF EXISTS SPACEXTABLE;
CREATE TABLE SPACEXTABLE AS
SELECT * FROM SPACEXTBL WHERE Date IS NOT NULL;
%%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE;

 * sqlite:///my_spacex.db
Done.
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql
SELECT DISTINCT Launch_Site FROM SPACEXTABLE;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


## 3) Quick Peek

In [12]:

%sql SELECT * FROM SPACEXTABLE LIMIT 5;

 * sqlite:///my_spacex.db
Done.


Date,Time (UTC),Booster_Version,Launch_Site,Payload,PAYLOAD_MASS__KG_,Orbit,Customer,Mission_Outcome,Landing_Outcome
2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of Brouere cheese",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


## 4) Distinct Launch Sites

In [13]:

%sql SELECT DISTINCT Launch_Site FROM SPACEXTABLE;

 * sqlite:///my_spacex.db
Done.


Launch_Site
CCAFS LC-40
VAFB SLC-4E
KSC LC-39A
CCAFS SLC-40


## 5) Launch Counts by Site

In [18]:
%%sql
SELECT Launch_Site, COUNT(*) AS Launch_Count
FROM SPACEXTABLE
GROUP BY Launch_Site
ORDER BY Launch_Count DESC;

 * sqlite:///my_spacex.db
Done.


Launch_Site,Launch_Count
CCAFS SLC-40,34
CCAFS LC-40,26
KSC LC-39A,25
VAFB SLC-4E,16


## 6) Min/Max Payload Mass (kg)

In [20]:
%%sql
-- column name is PAYLOAD_MASS__KG_
SELECT
  MIN(PAYLOAD_MASS__KG_) AS Min_Payload,
  MAX(PAYLOAD_MASS__KG_) AS Max_Payload
FROM SPACEXTABLE;

 * sqlite:///my_spacex.db
Done.


Min_Payload,Max_Payload
0,15600


## 7) Average Payload Mass by Orbit

In [22]:
%%sql
SELECT Orbit,
       ROUND(AVG(PAYLOAD_MASS__KG_),1) AS Avg_Payload_kg,
       COUNT(*) AS Missions
FROM SPACEXTABLE
GROUP BY Orbit
HAVING Missions >= 1
ORDER BY Avg_Payload_kg DESC;

 * sqlite:///my_spacex.db
Done.


Orbit,Avg_Payload_kg,Missions
Sub-orbital,12050.0,1
LEO,11063.5,25
Polar LEO,8070.0,8
GTO,5001.3,30
MEO,4340.7,3
LEO (ISS),3300.9,26
SSO,2825.8,6
HEO,466.0,2


## 8) Mission Outcome Counts per Launch Site

In [23]:
%%sql
SELECT Launch_Site,
       Mission_Outcome,
       COUNT(*) AS Outcome_Count
FROM SPACEXTABLE
GROUP BY Launch_Site, Mission_Outcome
ORDER BY Launch_Site, Outcome_Count DESC;

 * sqlite:///my_spacex.db
Done.


Launch_Site,Mission_Outcome,Outcome_Count
CCAFS LC-40,Success,25
CCAFS LC-40,Failure (in flight),1
CCAFS SLC-40,Success,32
CCAFS SLC-40,Success (payload status unclear),1
CCAFS SLC-40,Success,1
KSC LC-39A,Success,25
VAFB SLC-4E,Success,16


## 9) Derived Success Rate by Launch Site

In [24]:
%%sql sqlite:///my_spacex.db
SELECT Launch_Site,
       ROUND(AVG(CASE WHEN Mission_Outcome LIKE 'Success%' THEN 1 ELSE 0 END), 3) AS Success_Rate,
       COUNT(*) AS Missions
FROM SPACEXTABLE
GROUP BY Launch_Site
ORDER BY Success_Rate DESC;


Done.


Launch_Site,Success_Rate,Missions
VAFB SLC-4E,1.0,16
KSC LC-39A,1.0,25
CCAFS SLC-40,1.0,34
CCAFS LC-40,0.962,26


## 10) Launch Counts by Orbit

## 11) Top 5 Heaviest Payload Missions

In [25]:

%%sql 
SELECT Launch_Site, Booster_Version, Orbit, PAYLOAD_MASS__KG_ AS Payload_kg, Mission_Outcome
     FROM SPACEXTABLE
     WHERE PAYLOAD_MASS__KG_ IS NOT NULL
     ORDER BY Payload_kg DESC
     LIMIT 5;

 * sqlite:///my_spacex.db
Done.


Launch_Site,Booster_Version,Orbit,Payload_kg,Mission_Outcome
CCAFS SLC-40,F9 B5 B1048.4,LEO,15600,Success
CCAFS SLC-40,F9 B5 B1049.4,LEO,15600,Success
CCAFS SLC-40,F9 B5 B1051.3,LEO,15600,Success
CCAFS SLC-40,F9 B5 B1056.4,LEO,15600,Success
KSC LC-39A,F9 B5 B1048.5,LEO,15600,Success


## 12) Launch Counts by Year

In [26]:

%%sql 
SELECT SUBSTR(Date, 1, 4) AS Year, COUNT(*) AS Launches
     FROM SPACEXTABLE
     GROUP BY Year
     ORDER BY Year;

 * sqlite:///my_spacex.db
Done.


Year,Launches
2010,2
2012,2
2013,3
2014,6
2015,7
2016,8
2017,18
2018,20
2019,11
2020,24
